In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date
import re
cache = None
api = None
sync_date = None
def _download():
    global cache, api, sync_date
    if api is None:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    cache = api.download_issues(project_id="pbssoftware")
    sync_date = date.today()
    
def download(force=False):
    global cache, sync_date
    reason = None
    if cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif sync_date is None:
        reason = "can't determine last sync date"
    elif date.today() != sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        _download()
        filter_issues()

In [2]:
skip = {
    3057,  # 3.0 Web FRS
    3066,  # 3.0 Web FRS
    3194,  # URS
}
def relevant(i):
    if i.id in skip:
        return False
    if i.sprint_milestone != "3.0" or i.tracker != "Specification":
        return False
    if i.status in ("Rejected", "Closed"):
        return False
    return True

def filter_issues():
    global cache, issues_30
    issues_30 = [i for i in cache.values() if relevant(i)]
    print("Filtered %d relevant issues" % len(issues_30))
    
def get_issues(recache=False):
    download(recache)
    return issues_30.copy()

In [3]:
def check_toplevel():
    for i in get_issues():
        exp = "*+FRS%d+*" % i.id
        d = i.description
        if d.find(exp) == -1:
            if d.find("*+3.0WebFRS") > -1:
                print("Web FRS:", i.id)
            else:
                print("Not found:", i.id)
check_toplevel()

Filtered 80 relevant issues
Web FRS: 3308
Not found: 3466


In [4]:
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
def goddamnit():
    for i in get_issues():
        if not frs_re.search(i.description):
            if web_frs_re.search(i.description):
                print("Goddamnit Christian: %d" % i.id)
            else:
                print("Goddamnit Nathan: %d" % i.id)
goddamnit()

Goddamnit Nathan: 3466


In [5]:
def search(recache=False):
    compliant_search = re.compile(r"[\+\*]{1,2}(?:3\.0Web)?FRS[\d\.]*\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    def compliant(i):
        return i.id == 3194 or compliant_search(i.description)
    noncompliant = []; add = noncompliant.append
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add("Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    for msg in noncompliant:
        print(msg)

In [6]:
search()

Found 1 noncompliant issues
Goddamnit Nathan: 3466


In [8]:
compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}", re.MULTILINE).search
frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")

def compliant_description(d):
    return compliant_search(d) is not None
def compliant(i):
    return (i.id == 3194 or compliant_description(i.description))

def search(recache=False):
    compliant_search = re.compile(r"^[\+\*]{1,2}(?:3\.0Web)?FRS\d+\:?[\+\*]{1,2}").search
    frs_re = re.compile(r"[\*\+]+(3\.0Web)?(FRS[\d\.]*)\:?[\*\+]+")
    web_frs_re = re.compile(r"(3\.0WebFRS[\d\.]*)")
    n=0
    noncompliant = []
    def add(id, msg):
        noncompliant.append((id, msg))
    for i in get_issues(recache):
        if not compliant(i):
            #if not web_frs_re.search(i.description):
                add(i.id, "Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    noncompliant.sort(key=lambda arg:arg[0])
    for _, msg in noncompliant:
        print(msg)
search(False)

Found 1 noncompliant issues
Goddamnit Nathan: 3466


In [9]:
import clipboard
#print(compliant_description(clipboard.paste()))

In [6]:
toplevel_match = re.compile(r"^([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
header_match = re.compile(r"^([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match
valid_headers = ["User Requirement", "James's Notes", "Original FRS", "URS","Action", 
                 "User Requirements","Notes", "Note", "Implementation","Implementation Details",
                 "Current Behavior", "Background", "Past Behavior", "Release Notes"]
subitem_match = re.compile(r"^([\+\*]+)\s\*(FRS)([\d\.]+)\:\*(.+)$").match
list_match = re.compile(r"([\#\*]+)\s(.*)").match
img_match = re.compile(r"^!(.+)!$").match
captions = ["Image"]
caption_match = re.compile(r"\*((?:%s) \d+)\*\:(.+)$" % "|".join(captions)).match
note_match = re.compile(r"\*?(Note|Notes)\:?\*\:?").match
maybe_frs = re.compile(r"(FRS\d[\d\.]+)").search
webfrs_match = re.compile(r"^[\*\+]+(3\.0WebFRS\d+)[\*\+]+").match

TOPLEVEL = 0
HEADER = 1
SUBITEM = 2
OTHER = 3
LISTITEM = 4
EMPTY = 5
IMG = 6
CAPTION = 7
NOTE = 8
MAYBE_FRS = 9
WEB_FRS = 10

def compliant_line(i, l):
    if not l.strip():
        return True, EMPTY, ""
    
    m = toplevel_match(l)
    if m:
        pre, frs, post, other = m.groups()
        if pre != "*+" or post != "+*": return False, TOPLEVEL, "Toplevel: pre != post: %r" % l
        if frs[3:] != str(i.id): return False, TOPLEVEL, "FRS ID mismatch"
        if other.strip(): return False, TOPLEVEL, "Tail: %r" % l
        return True, TOPLEVEL, ""
    
        
    if webfrs_match(l):
        return True, WEB_FRS, ""
    
    m = header_match(l)
    if m:
        pre, header, icol, post, ocol, other = m.groups()
        if pre != post[::-1]: return False, HEADER, "Header: pre != post: %r and %r" % (pre, post)
        if header not in valid_headers: 
            # Special cases
            if header.startswith("Implementation"): return True, HEADER, ""
            if header.startswith("Original FRS"): return True, HEADER, ""
            return False, HEADER, "Invalid header: %r" % header
        if other.strip(): print("Warning, found non-empty other in header")
        return True, HEADER, ""
    
    m = subitem_match(l)
    if m:
        ind, frs, nums, other = m.groups()
        if len(ind) != len(nums.split("."))-1: return False, SUBITEM, "Wrong Indent"
        if not other: return False, SUBITEM, "No text for subitem"
        return True, SUBITEM, ""
    
    m = list_match(l)
    if m:
        ind, other = m.groups()
        c = ind[0]
        if not all(ch == c for ch in ind): return False, LISTITEM, "wtf"
        return True, LISTITEM, ""
    
    m = img_match(l)
    if m:
        return True, IMG, ""
    
    m = caption_match(l)
    if m:
        return True, CAPTION, ""
    
    m = note_match(l)
    if m:
        return True, NOTE, ""
    
    if maybe_frs(l):
        # special case
        if l.startswith("*Original FRS"): return True, HEADER, ""
        return False, MAYBE_FRS, "MaybeFRS: %r" % l
    
    return True, OTHER, "Other: %r" % l

def consume_pre(it):
    level = 1
    while level > 0:
        line = next(it)
        if "<pre>" in line:
            level += 1
        elif "</pre>" in line:
            level -= 1
    return line

def compliant_description(i, d):
    rv = True
    it = iter(d.splitlines())
    while True:
        line = next(it, None)
        if line is None:
            break
        if "<pre>" in line:
            line = consume_pre(it)
            if line is None:
                break
            
        comp, typ, msg = compliant_line(i, line)
        if not comp:
            print(i.id, msg)
            rv = False
    return rv
    
def compliant(i):
    return (i.id == 3194 or compliant_description(i, i.description))

def search(recache=False):
    n=0
    noncompliant = []
    def add(id, msg):
        noncompliant.append((id, msg))
    for i in get_issues(recache):
        if not compliant(i):
            add(i.id, "Goddamnit Nathan: %d" % i.id)

    print("Found %d noncompliant issues" % len(noncompliant))
    noncompliant.sort(key=lambda arg:arg[0])
    for _, msg in noncompliant:
        print(msg)
search(False)

Warning, found non-empty other in header
3402 Tail: '*+FRS3402+*: If agitation is in auto mode when the comb plate is triggered, it should resume agitation at 0% duty when the comb plate is reconnected. '
1073 MaybeFRS: 'All FRS with the form "FRS1073.0X" have actions which are in the Shell'
1073 MaybeFRS: 'All FRS with the form "FRS1073.1X" use the Webservice Event to DB sub vi'
1073 MaybeFRS: 'All FRS with the form "FRS1073.13X" use the Webservice Event to DB sub vi and are in the Webservice Command Handler'
2808 Invalid header: 'Notes on other, rejected methods'
1510 Invalid header: 'Desired behavior'
Found 4 noncompliant issues
Goddamnit Nathan: 1073
Goddamnit Nathan: 1510
Goddamnit Nathan: 2808
Goddamnit Nathan: 3402


In [11]:
m1 = re.compile(r"\*\+(.+)\+\*").match
m2 = re.compile(r"(FRS\d+)").search
m3 = None
for i in get_issues():
    d = i.description
    for l in d.splitlines():
        if not m1(l):
            if m2(l):
                try:
                    a, _= l.split(" ", 1)
                except:
                    pass
                else:
                    if a.count("*") != len(a):
                        print(l)
         

All FRS with the form "FRS1073.0X" have actions which are in the Shell
All FRS with the form "FRS1073.1X" use the Webservice Event to DB sub vi
All FRS with the form "FRS1073.13X" use the Webservice Event to DB sub vi and are in the Webservice Command Handler
*Original FRS1073.01*
*Original FRS1073.02*
*Original FRS1073.03*
*Original FRS1073.11*
*Original FRS1073.1304*
*Original FRS1073.1305*
*Original FRS1073.1306*
*Original FRS1073.1307*
*Original FRS1073.1308*
*Original FRS1073.1309*
*Original FRS1073.1310*
*Original FRS1073.1311*
*Original FRS1073.1312*
*Original FRS1073.1313*
*Original FRS1073.1314*
*Original FRS1073.1315*
*Original FRS1073.1316*


In [7]:
import os
toplevel_match = re.compile(r"^\>?([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
subitem_match = re.compile(r"^\>?([\+\*]+)\s\*(FRS)([\d\.]+)\:\*(.+)$").match
header_match = re.compile(r"^\>?([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match

def extract_descr(d):
    it = iter(d.strip().splitlines())
    frs = []
    m = None
    while not m:
        line = next(it, None)
        if line is None:
            return frs
        m = toplevel_match(line)
        if m:
            while True:
                frs.append(line)
                line = next(it, None)
                if line is None:
                    break
                if header_match(line):
                    break
    return frs
                

def write_frs(f, frs):
    for i, lines in frs:
        tl = lines[0]
        f.write(tl + " %s (#%d)\n" % (i.subject, i.id))
        for line in lines[1:]: f.write(line+"\n")
        f.write("\n\n")
        
def write_frs2(f, frs):
    w = f.write
    for i, lines in frs:
        w("*+Issue #%d+*: %s\n" % (i.id, i.subject))
        for line in lines:
            w(">"+line+"\n")
        w("\n\n")
    
def extract_frs():
    frs = []
    for i in get_issues():
        lines = extract_descr(i.description)
        if not lines: continue
        frs.append((i, lines))
    frs.sort(key=lambda v:v[0].id)
    return frs
        

def descr_nums(desc):
    nums = []
    for l in desc.splitlines():
        m = toplevel_match(l)
        if m:
            _, frs, _, _ = m.groups()
            nums.append(frs)
            continue
        m = subitem_match(l)
        if m:
            _, frs, ns, _ = m.groups()
            nums.append(frs+ns)
            continue
    return nums


def all_frs_numbers():
    nums = []
    for i in get_issues():
        nums.extend(descr_nums(i.description))
    # nums.sort(key=lambda v: tuple(int(i) for i in v[3:].split(".")))
    return nums

def verify_frs_numbers(): 
    """
    Verify the script found all FRS lines for the compiled
    FRS, by first extracting all possible lines directly from
    issue descriptions, then scanning the extracted FRS lines. 
    """
    fn = "full_frs.txt"
    frs = extract_frs()
    with open(fn, "w") as f:
        write_frs2(f, frs)
          
    with open(fn, 'r') as f:
        txt = f.read()
    
    nums = set(all_frs_numbers())
    nums2 = descr_nums(txt)
    nums2 = set(nums2)
    if nums != nums2:
        key = lambda s: tuple(int(i) for i in s[3:].split("."))
        print("Missing items: \n%s" % ("\n".join(sorted(nums-nums2, key=key))))
        print("Extra Items: %s" % ", ".join(nums2 - nums))
        print("verify_frs_numbers: Failure")
    else:
        print("verify_frs_numbers: Success")

verify_frs_numbers()
                
                

verify_frs_numbers: Success


In [8]:
import os
os.startfile("full_frs.txt")